This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
!pip install -U chromadb pandas
!pip install -U langchain langchain-community langchain-openai fastapi

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [6]:
import os
import json
import pandas as pd
import chromadb
from io import StringIO


from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain
#from langchain.output_parsers import PydanticOutputParser
from fastapi.encoders import jsonable_encoder
from langchain.schema import SystemMessage, HumanMessage
#from langchain_core.output_parsers import StrOutputParser

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"
client = chromadb.PersistentClient(path="./chroma_db") # Stores data in a local directory


model_name = "gpt-3.5-turbo-instruct"
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens = 5000, api_key="OPENAI_API_KEY")
instruction = """
Generate 25 Real Estate lisitngs in CSV format with the following fields: Neighborhood, Price, Bedrooms, Bathrooms, House Size, Description.
Follow this format
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. 
This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels 
and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful 
hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious 
backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without 
compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to 
organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park
or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, 
commuting is a breeze.
"""

try:
    RE_listings = llm.invoke(instruction).content
except Exception as e:
    # In case quota not enough
    raise RuntimeError(
        f"LLM call failed (Please check quota). Underlying error: {e}"
    )


# Convert RE_listings->DataFrame
df = pd.read_csv(StringIO(RE_listings), sep=';')
df.to_csv('RE_listings.csv', index=False)

# Load CSV
csv_loader = CSVLoader(file_path='./RE_listings.csv')
RE_listing_docs = csv_loader.load()

# Embeddings created
embeddings = OpenAIEmbeddings(api_key="OPENAI_API_KEY")

# Build Chroma Database
db = Chroma.from_documents(
    documents=RE_listing_docs,
    embedding=embeddings,
    collection_name="RE_listings",
    persist_directory="data",
)
db.persist()

# Test User Input
user_input = input("Welcome to House Match; Please enter what you are looking for: ")
print("You entered:", user_input)

# Retrieval 
results = db.similarity_search(user_input, k=5)
for i, d in enumerate(results, 1):
    print(f"\nResult {i}:\n{d.page_content}\nMetadata: {d.metadata}") 

/tmp/ipykernel_346/1070632236.py:29: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens = 5000, api_key="voc-11912945411266772925473684a296746b7b2.99491625")


RuntimeError: LLM call failed (Please check quota). Underlying error: 'NoneType' object is not iterable